<a href="https://colab.research.google.com/github/tvani2/Neural-Networks-Facial-Expression-Recognition-Challenge/blob/main/Expr_recognition_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle wandb onnx -Uq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 35.1 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
! mkdir ~/.kaggle

In [5]:
!cp /content/drive/MyDrive/cs231n/assignments/assignment4/kaggle.json ~/.kaggle/kaggle.json

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

 88% 250M/285M [00:00<00:00, 816MB/s] 
100% 285M/285M [00:04<00:00, 68.7MB/s]


In [8]:
! unzip challenges-in-representation-learning-facial-expression-recognition-challenge

Archive:  challenges-in-representation-learning-facial-expression-recognition-challenge.zip
  inflating: example_submission.csv  
  inflating: fer2013.tar.gz          
  inflating: icml_face_data.csv      
  inflating: test.csv                
  inflating: train.csv               


In [44]:
import wandb
wandb.login()

True

In [63]:
import wandb

run = wandb.init(
    project="Facial_Expression_Recognition_1",
    config={
        "architecture": "ImprovedCNN_Combined",
        "num_classes": 7,
        "dropout_prob": 0.3,
        "learning_rate": 0.001,
        "optimizer": "Adam",
        "scheduler": "ReduceLROnPlateau",
        "scheduler_patience": 5,
        "scheduler_factor": 0.1,
        "loss_function": "CrossEntropyLoss",
        "batch_size": 64,
        "epochs": 20,
        "input_size": (48, 48)
    }
)

In [64]:
import wandb
import os
import pandas as pd
# Create a wandb artifact
# Reference [1] explains how to create an artifact with a type and project
raw_data_artifact = wandb.Artifact(
    name="facial-expression-dataset",
    type="dataset",
    description="Facial Expression Recognition Challenge Dataset"
)
# Log the artifact to the current run
wandb.log_artifact(raw_data_artifact)

print("Data loaded and logged as a wandb artifact.")

Data loaded and logged as a wandb artifact.


In [65]:
import torch # Main PyTorch Library
from torch import nn # Used for creating the layers and loss function
from torch.optim import Adam # Adam Optimizer
import torchvision.transforms as transforms # Transform function used to modify and preprocess all the images
from torch.utils.data import Dataset, DataLoader # Dataset class and DataLoader for creating the objects
from sklearn.preprocessing import LabelEncoder # Label Encoder to encode the classes from strings to numbers
import matplotlib.pyplot as plt # Used for visualizing the images and plotting the training progress
from PIL import Image # Used to read the images from the directory
import pandas as pd # Used to read/create dataframes (csv) and process tabular data
import numpy as np # preprocessing and numerical/mathematical operations
import os # Used to read the images path from the directory

device = "cuda" if torch.cuda.is_available() else "cpu" # detect the GPU if any, if not use CPU, change cuda to mps if you have a mac
print("Device available: ", device)

Device available:  cuda


In [66]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the dataset
# Change the file path to the correct location after unzipping
df = pd.read_csv('./train.csv')
X = df['pixels']
y = df['emotion']

train_size = 0.70
val_size = 0.15
test_size = 0.15
X_temp, X_test_new, y_temp, y_test_new = train_test_split(
    X, y, test_size=test_size, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=(val_size / (train_size + val_size)), random_state=42, stratify=y_temp
)

# --- Convert pandas Series of string pixels to numerical NumPy array ---
# Function to convert a space-separated string of pixels into a NumPy array and reshape
def process_pixels_string(pixel_string):
    return np.array(pixel_string.split(), dtype=np.uint8).reshape(48, 48)

# Apply the function to each element in the Series and stack them
# This will create a 3D numpy array where each element is a 48x48 image
X_train_np = np.stack(X_train.apply(process_pixels_string).values)
X_val_np = np.stack(X_val.apply(process_pixels_string).values)
X_test_new_np = np.stack(X_test_new.apply(process_pixels_string).values)

# Expand dimensions to add a channel dimension (for grayscale images)
X_train_np = np.expand_dims(X_train_np, axis=-1)
X_val_np = np.expand_dims(X_val_np, axis=-1)
X_test_new_np = np.expand_dims(X_test_new_np, axis=-1)

# --- Normalization Function ---
def normalize_pixels(image_array):
    """
    Scales pixel values from [0, 255] to [0, 1].

    Args:
        image_array (numpy.ndarray): Input image data (can be 2D or 3D/4D array).

    Returns:
        numpy.ndarray: Normalized image data.
    """
    # Ensure the data type is float before division
    # Dividing uint8 by an integer might perform integer division.
    # Dividing uint8 by a float results in float.
    return image_array / 255.0

# --- Apply Normalization ---
X_train_normalized = normalize_pixels(X_train_np)
X_val_normalized = normalize_pixels(X_val_np)
X_test_new_normalized = normalize_pixels(X_test_new_np)

In [67]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split

# --- Convert Normalized NumPy Arrays and Labels to Tensors ---

def convert_to_tensors(image_arrays, labels):

    image_tensors = torch.tensor(image_arrays, dtype=torch.float32).permute(0, 3, 1, 2)

    # Convert labels (which might be a pandas Series) to a NumPy array first, then to long tensor.
    # torch.long is used for classification labels in CrossEntropyLoss.
    if isinstance(labels, pd.Series):
        label_tensors = torch.tensor(labels.values, dtype=torch.long)
    else: # Assuming it's already a numpy array
         label_tensors = torch.tensor(labels, dtype=torch.long)


    return image_tensors, label_tensors

# --- Apply Conversion to your Data Splits ---

X_train_tensor, y_train_tensor = convert_to_tensors(X_train_normalized, y_train)
X_val_tensor, y_val_tensor = convert_to_tensors(X_val_normalized, y_val)
X_test_new_tensor, y_test_new_tensor = convert_to_tensors(X_test_new_normalized, y_test_new)


In [68]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# --- Create Custom Dataset Class ---

class EmotionDataset(Dataset):
    """Custom Dataset for facial emotion data."""

    def __init__(self, images, labels, transform=None):
        """
        Args:
            images (torch.Tensor): Tensor of images (N, C, H, W).
            labels (torch.Tensor): Tensor of labels (N,).
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels) # The number of samples

    def __getitem__(self, idx):
        # Get image and label for the given index
        image = self.images[idx]
        label = self.labels[idx]

        # Apply transform if any (though we've already normalized,
        # this is where augmentation would typically go for train)
        if self.transform:
             pass

        return image, label

# --- Create Datasets ---
train_dataset = EmotionDataset(X_train_tensor, y_train_tensor, transform=None)
val_dataset = EmotionDataset(X_val_tensor, y_val_tensor, transform=None)
test_new_dataset = EmotionDataset(X_test_new_tensor, y_test_new_tensor, transform=None)

In [69]:
batch_size = 64 # A common starting batch size

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# val_dataloader: shuffle=False - order doesn't matter for evaluation
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# test_new_dataloader: shuffle=False - order doesn't matter for final evaluation
test_new_dataloader = DataLoader(test_new_dataset, batch_size=batch_size, shuffle=False)

In [70]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ImprovedCNN_Combined(nn.Module):

    def __init__(self, num_classes=7, dropout_prob=0.3):
        super(ImprovedCNN_Combined, self).__init__()

        # Block 1: Conv -> BatchNorm -> ReLU -> Pool
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2) # Output: 32x24x24

        # Block 2: Conv -> BatchNorm -> ReLU -> Pool
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # Output: 64x12x12

        # Block 3: Conv -> BatchNorm -> ReLU -> Pool
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2) # Output: 128x6x6

        # Flattened size: 128 * 6 * 6 = 4608
        # Fully connected layers with Dropout
        self.fc1 = nn.Linear(128 * 6 * 6, 512) # Added an intermediate linear layer
        self.dropout1 = nn.Dropout(dropout_prob)

        self.fc2 = nn.Linear(512, num_classes) # Final classification layer

    def forward(self, x):
        # Block 1
        x = self.pool1(F.relu(self.bn1(self.conv1(x)))) # -> N x 32 x 24 x 24

        # Block 2
        x = self.pool2(F.relu(self.bn2(self.conv2(x)))) # -> N x 64 x 12 x 12

        # Block 3
        x = self.pool3(F.relu(self.bn3(self.conv3(x)))) # -> N x 128 x 6 x 6

        # Flatten
        x = x.view(-1, 128 * 6 * 6) # -> N x 4608

        # Fully connected layers with ReLU and Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)

        x = self.fc2(x) # Output layer (no activation, CrossEntropyLoss handles Softmax)
        return x

In [71]:
num_classes = 7
model = ImprovedCNN_Combined(num_classes=num_classes, dropout_prob=0.3)

model.to(device)
print(f"Model moved to: {device}")
wandb.watch(model, log="all")

Model moved to: cuda


In [72]:
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
from sklearn.metrics import accuracy_score

# Loss function
criterion = nn.CrossEntropyLoss()

# Learning rate and optimizer
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.1,
    patience=5,
    verbose=True
)

# Training function
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / total_samples
    epoch_accuracy = correct_predictions / total_samples

    print(f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}")
    return epoch_loss, epoch_accuracy

# Validation function
def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / total_samples
    epoch_accuracy = correct_predictions / total_samples

    print(f"Val Loss: {epoch_loss:.4f}, Val Accuracy: {epoch_accuracy:.4f}")
    return epoch_loss, epoch_accuracy

# Main training loop
num_epochs = 20
print(f"\nStarting training for {num_epochs} epochs...\n")
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss, train_accuracy = train_epoch(model, train_dataloader, criterion, optimizer, device)
    val_loss, val_accuracy = validate_epoch(model, val_dataloader, criterion, device)

    # Log metrics to wandb
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_accuracy": train_accuracy,
        "val_loss": val_loss,
        "val_accuracy": val_accuracy
    })

    scheduler.step(val_loss)

print("\nTraining finished.")

# Final test evaluation
def evaluate_model(model, dataloader, criterion, device, set_name="Test Set"):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    avg_loss = running_loss / total_samples
    accuracy = correct_predictions / total_samples

    print(f"\n{set_name} Results:")
    print(f"  Loss: {avg_loss:.4f}")
    print(f"  Accuracy: {accuracy:.4f}")

    # Log test metrics to wandb
    wandb.log({
        f"{set_name.lower().replace(' ', '_')}_loss": avg_loss,
        f"{set_name.lower().replace(' ', '_')}_accuracy": accuracy
    })

# Evaluate on the test set
evaluate_model(model, test_new_dataloader, criterion, device, set_name="New Test Set (from original train data)")

# Finish wandb run
wandb.finish()

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(



Starting training for 20 epochs...

Epoch 1/20
Train Loss: 1.7616, Train Accuracy: 0.3077
Val Loss: 1.5215, Val Accuracy: 0.4117
Epoch 2/20
Train Loss: 1.4679, Train Accuracy: 0.4351
Val Loss: 1.3501, Val Accuracy: 0.4671
Epoch 3/20
Train Loss: 1.3543, Train Accuracy: 0.4801
Val Loss: 1.2852, Val Accuracy: 0.5120
Epoch 4/20
Train Loss: 1.2744, Train Accuracy: 0.5136
Val Loss: 1.2377, Val Accuracy: 0.5161
Epoch 5/20
Train Loss: 1.2217, Train Accuracy: 0.5391
Val Loss: 1.2389, Val Accuracy: 0.5236
Epoch 6/20
Train Loss: 1.1594, Train Accuracy: 0.5582
Val Loss: 1.1778, Val Accuracy: 0.5493
Epoch 7/20
Train Loss: 1.1159, Train Accuracy: 0.5709
Val Loss: 1.1790, Val Accuracy: 0.5635
Epoch 8/20
Train Loss: 1.0514, Train Accuracy: 0.5997
Val Loss: 1.1781, Val Accuracy: 0.5526
Epoch 9/20
Train Loss: 0.9913, Train Accuracy: 0.6231
Val Loss: 1.1715, Val Accuracy: 0.5628
Epoch 10/20
Train Loss: 0.9335, Train Accuracy: 0.6391
Val Loss: 1.1603, Val Accuracy: 0.5654
Epoch 11/20
Train Loss: 0.8691, 

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
new_test_set_(from_original_train_data)_accuracy,▁
new_test_set_(from_original_train_data)_loss,▁
train_accuracy,▁▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇████
train_loss,█▇▆▆▅▅▅▅▄▄▄▃▃▃▂▂▁▁▁▁
val_accuracy,▁▃▅▅▅▆▇▆▇▇▇▇▇▇▇▇████
val_loss,█▅▃▃▃▁▁▁▁▁▂▂▂▄▄▅▆▆▇▇
epoch,20
new_test_set_(from_original_train_data)_accuracy,0.60065
new_test_set_(from_original_train_data)_loss,1.45833
train_accuracy,0.87594


In [73]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [74]:
import wandb

run = wandb.init(
    project="Facial_Expression_Recognition_1",
    config={
        "architecture": "ImprovedCNN_Combined",
        "num_classes": 7,
        "dropout_prob": 0.4,
        "learning_rate": 0.0005,
        "optimizer": "Adam",
        "weight_decay": 5e-3,
        "scheduler": "ReduceLROnPlateau",
        "scheduler_patience": 5,
        "scheduler_factor": 0.1,
        "loss_function": "CrossEntropyLoss",
        "batch_size": 64,
        "epochs": 20,
        "input_size": (48, 48)
    }
)

In [78]:
num_classes = 7
model = ImprovedCNN_Combined(num_classes=num_classes, dropout_prob=0.4)

model.to(device)
print(f"Model moved to: {device}")
wandb.watch(model, log="all")

Model moved to: cuda


In [79]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
import wandb

# Initialize wandb
wandb.init(project="Facial_Expression_Recognition_1", name="early stopping + modifications")  # Change project and run names as needed

# Loss function
criterion = nn.CrossEntropyLoss()

# Learning rate and optimizer
learning_rate = 0.0005
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-3)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.1,
    patience=5,
    verbose=True
)

# Training function
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / total_samples
    epoch_accuracy = correct_predictions / total_samples

    print(f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}")
    return epoch_loss, epoch_accuracy

# Validation function
def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    epoch_loss = running_loss / total_samples
    epoch_accuracy = correct_predictions / total_samples

    print(f"Val Loss: {epoch_loss:.4f}, Val Accuracy: {epoch_accuracy:.4f}")
    return epoch_loss, epoch_accuracy

# Main training loop
num_epochs = 20
print(f"\nStarting training for {num_epochs} epochs...\n")
early_stopping = EarlyStopping(patience=5, min_delta=1e-3)

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss, train_accuracy = train_epoch(model, train_dataloader, criterion, optimizer, device)
    val_loss, val_accuracy = validate_epoch(model, val_dataloader, criterion, device)
    scheduler.step(val_loss)

    # Log to wandb
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_accuracy": train_accuracy,
        "val_loss": val_loss,
        "val_accuracy": val_accuracy,
        "learning_rate": optimizer.param_groups[0]['lr']
    })

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered.")
        break

print("\nTraining finished.")

# Final test evaluation
def evaluate_model(model, dataloader, criterion, device, set_name="Test Set"):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    avg_loss = running_loss / total_samples
    accuracy = correct_predictions / total_samples

    print(f"\n{set_name} Results:")
    print(f"  Loss: {avg_loss:.4f}")
    print(f"  Accuracy: {accuracy:.4f}")
    wandb.log({
        f"{set_name.lower().replace(' ', '_')}_loss": avg_loss,
        f"{set_name.lower().replace(' ', '_')}_accuracy": accuracy
    })

# Call test evaluation
evaluate_model(model, test_new_dataloader, criterion, device, set_name="New Test Set (from original train data)")

# Finish wandb run
wandb.finish()

epoch,▁█
learning_rate,▁▁
train_accuracy,▁█
train_loss,█▁
val_accuracy,█▁
val_loss,█▁
epoch,2
learning_rate,0.0005
train_accuracy,0.76546
train_loss,0.65009
val_accuracy,0.55398



Starting training for 20 epochs...

Epoch 1/20
Train Loss: 1.6518, Train Accuracy: 0.3530
Val Loss: 1.5237, Val Accuracy: 0.3938
Epoch 2/20
Train Loss: 1.4291, Train Accuracy: 0.4519
Val Loss: 1.4024, Val Accuracy: 0.4360
Epoch 3/20
Train Loss: 1.3327, Train Accuracy: 0.4899
Val Loss: 1.2654, Val Accuracy: 0.5140
Epoch 4/20
Train Loss: 1.2800, Train Accuracy: 0.5118
Val Loss: 1.2506, Val Accuracy: 0.5236
Epoch 5/20
Train Loss: 1.2362, Train Accuracy: 0.5324
Val Loss: 1.2519, Val Accuracy: 0.5326
Epoch 6/20
Train Loss: 1.1991, Train Accuracy: 0.5439
Val Loss: 1.1802, Val Accuracy: 0.5454
Epoch 7/20
Train Loss: 1.1633, Train Accuracy: 0.5607
Val Loss: 1.1911, Val Accuracy: 0.5375
Epoch 8/20
Train Loss: 1.1312, Train Accuracy: 0.5731
Val Loss: 1.1560, Val Accuracy: 0.5589
Epoch 9/20
Train Loss: 1.1010, Train Accuracy: 0.5868
Val Loss: 1.2145, Val Accuracy: 0.5410
Epoch 10/20
Train Loss: 1.0773, Train Accuracy: 0.5965
Val Loss: 1.2003, Val Accuracy: 0.5373
Epoch 11/20
Train Loss: 1.0425, 

epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
new_test_set_(from_original_train_data)_accuracy,▁
new_test_set_(from_original_train_data)_loss,▁
train_accuracy,▁▃▄▄▅▅▆▆▆▆▇▇▇▇██
train_loss,█▆▅▅▄▄▄▃▃▃▂▂▂▂▁▁
val_accuracy,▁▃▆▆▇▇▇█▇▇█▇██▇▇
val_loss,█▆▄▃▃▂▂▂▃▂▁▂▁▂▂▂
epoch,16
learning_rate,0.0005
new_test_set_(from_original_train_data)_accuracy,0.55096
